# Sentiment Analysis using finetuned BERT model on financial documents 10K


In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
#mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import json 
traind=[]
vald=[]
dir = "/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work/data/"
# Opening JSON file 
with open(dir+'FinNum_training_v3.json',) as f: 
  traind = json.load(f) 
with open(dir+'FinNum_dev_v3.json',) as f: 
  vald = json.load(f) 

train = pd.DataFrame(traind) 
val = pd.DataFrame(vald) 
print(train.shape, val.shape)


(7187, 5) (1044, 5)


In [4]:
train.head(2)

,tweet,target_num,offset,target_cashtag,relation
0,"$XXII Scott Gottlieb, Commissioner of FDA spee...",3,74,XXII,1
1,$TSLA Maybe they calculate that 7ct energy pri...,7,32,TSLA,1


In [5]:
val.head(2)

,tweet,target_num,offset,target_cashtag,relation
0,$CAT My ST alert at $121 now up a nice 19% if ...,121,21,CAT,1
1,$CYTR Wondering if i should rather add Jul/Sep...,1,47,CYTR,1


In [6]:
train.info(), val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7187 entries, 0 to 7186
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet           7187 non-null   object
 1   target_num      7187 non-null   object
 2   offset          7187 non-null   int64 
 3   target_cashtag  7187 non-null   object
 4   relation        7187 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 280.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet           1044 non-null   object
 1   target_num      1044 non-null   object
 2   offset          1044 non-null   int64 
 3   target_cashtag  1044 non-null   object
 4   relation        1044 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 40.9+ KB


(None, None)

In [7]:
cols = ['tweet', 'relation']
train = train[cols]
val = val[cols]

In [8]:
print(train.relation.value_counts())
print(val.relation.value_counts())

1    5827
0    1360
Name: relation, dtype: int64
1    850
0    194
Name: relation, dtype: int64


In [9]:
label_dict = {}
for index, possible_label in enumerate(train.relation.unique()):
    label_dict[possible_label] = index

In [10]:
label_dict, len(label_dict)

({0: 1, 1: 0}, 2)

## Training/Validation Split

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_val, y_train, y_val = train['tweet'], val['tweet'], train['relation'], val['relation']

## Loading Tokenizer and Encoding our Data

In [13]:
!pip install transformers

In [14]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [15]:
modeldir = "/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work/tenkBERTv2Cased/"

In [16]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')


In [17]:
encoded_data_train = tokenizer.batch_encode_plus(
    train.tweet.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    val.tweet.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train.relation.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(val.relation.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [18]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [19]:
len(dataset_train), len(dataset_val)

(7187, 1044)

## Setting up BERT Pretrained Model

In [20]:
from transformers import BertForSequenceClassification

In [21]:
model = BertForSequenceClassification.from_pretrained(modeldir,
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)



Some weights of the model checkpoint at /content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work/tenkBERTv2Cased/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not

## Creating Data Loaders

In [22]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [23]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

## Setting Up Optimiser and Scheduler

In [24]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [25]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [26]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Defining our Performance Metrics

*Accuracy* metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [27]:
import numpy as np

In [28]:
from sklearn.metrics import f1_score

In [29]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [30]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [31]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [33]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [34]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.383646958205435
Validation loss: 0.20981672857746933
F1 Score (Weighted): 0.9166013887035218



Epoch 2
Training loss: 0.2923223298788071
Validation loss: 0.2605918704560309
F1 Score (Weighted): 0.8932083952430432



Epoch 3
Training loss: 0.23626761219567724
Validation loss: 0.2855960011595126
F1 Score (Weighted): 0.8893484060869805



In [35]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [36]:
accuracy_per_class(predictions, true_vals)

Class: 1
Accuracy: 94/194

Class: 0
Accuracy: 849/850

